## List Canvas items
Change the url to list your required items and write them to excel 
https://rmit.instructure.com/doc/api/live

In [ ]:
#==========================================================================#
# list-canvas-items
#==========================================================================#
# * Template to list canvas items, refer to the Canvas Live API for urls 
#
# Cathy Leahy
# Last update: 11/05/2018
#--------------------------------------------------------------------------#
# Setup
# [1] config
# 
#
#--------------------------------------------------------------------------#
# TO DO 
# * improve error handeling
#--------------------------------------------------------------------------#
import csv, requests, time
from time import gmtime, strftime
from pprint import pprint
import json
import pandas as pd


#==========================================================================#
# Variables
#==========================================================================#
# Adapted from G. Q. Maguire Jr.
with open('../config/config.json') as json_data_file:
    configuration = json.load(json_data_file)
    access_token=configuration["canvas"]["access_token"]

header = {'Authorization' : 'Bearer ' + access_token}
payload = {}
baseURL="https://"+configuration["canvas"]["host"]+"/api/v1/"
log_file = 'log.txt' # timestamped log file

#==========================================================================#
# Main
#==========================================================================#
def main():
    printDate = strftime("%Y-%m-%d", time.localtime())  
    filename = 'canvas-report.xlsx'
    
    course_id = input("Enter Course ID: ") 
    url='{}courses/{}/users'.format(baseURL, course_id) # course user list
    #url='{}courses/{}/pages'.format(baseURL, course_id) # course page list
    
    
    canvas_list = get_items(url)
    write_to_excel(canvas_list, 'Sheet 1', filename)   
    print_log('End      ')


## ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
## API call to get Canvas items
## ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Adapted from G. Q. Maguire Jr.
def get_items(url):
    items =[]
    items_json = []
    
    r = requests.get(url, headers = header)   
    if r.status_code == requests.codes.ok:
        items_json = r.json()
    
    for item in items_json:  
        items.append(item)
      
    while r.links['current']['url'] != r.links['last']['url']:
            r = requests.get(r.links['next']['url'], headers=header)
            items_json = r.json()
            for item in items_json:
                items.append(item)

    return items

## ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
## Write json data to an excel file
## ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Adapted from G. Q. Maguire Jr.
def write_to_excel(data, tab, file):
    data_df = pd.io.json.json_normalize(data)
    try:
        writer = pd.ExcelWriter(file)
        data_df.to_excel(writer, sheet_name=tab)
        writer.save()
    except:
        # need errorhandler - file open
        print("Error writing to file:")

## ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
## Write message to the log file
## ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Adapted from G. Q. Maguire Jr.
def print_log(msg):    
    log_time = str(time.asctime(time.localtime(time.time())))   
    message = "{}: {}".format(msg, log_time)
    write_to_log(message) 
    
def write_to_log(message):
    with open(log_file, 'a') as log:               
        log.write(message + "\n")     # write to file              
        pprint(message)               # print to screen
        
    
    
if __name__ == "__main__": main()